In [1]:
from fooof import FOOOF
from src.utils import get_bids_file
from src.params import RESULT_PATH, PREPROC_PATH, SUBJ_CLEAN, ACTIVE_RUN, PASSIVE_RUN
import mne
import numpy as np
from fooof.utils.download import load_fooof_data
import matplotlib.pyplot as plt

In [17]:
task = 'LaughterActive'
conditions = ['LaughReal']
all_psd = []

if task == 'LaughterActive':
    run_list = ACTIVE_RUN
elif task == 'LaughterPassive':
    run_list = PASSIVE_RUN

for i, cond in enumerate(conditions) :
    print('condition -->', cond)
    list_all_data = []
    list_epochs_ave = []
    freqs_list = []

    for subj in ['01', '02'] :
        epochs_time = []
        list_epochs = []

        for run in run_list : 
            _, psd_path = get_bids_file(PREPROC_PATH, subj=subj, task=task, run=run, stage='proc-clean_epo')
            epochs = mne.read_epochs(psd_path)
            epochs.pick_types(meg=True, ref_meg=False,  exclude='bads')
            epochs = epochs.apply_baseline(baseline=(None, 0))
            
            psd = epochs[cond].compute_psd(fmin=2.0, fmax=40.0)

            freqs_subj = psd.freqs
            freqs_list.append(freqs_subj)
            
            data = psd.get_data()
            all_psd.append(data)
            data_ave_epochs = np.mean(data, axis = 0)

            # Average frequency across subjects
freqs = np.mean(np.array(freqs_list), axis=0)
print(freqs.shape)

# Average power across :
# runs
psd_ave_run = np.mean(np.array(all_psd), axis=0)
# epochs
psd_ave_epo = np.mean(np.array(psd_ave_run), axis=0)
# channels
psd = np.mean(np.array(psd_ave_epo), axis=0)

print(psd.shape)

condition --> LaughReal
Reading /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_proc-clean_epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -500.00 ...    1500.00 ms
        5 CTF compensation matrices available
Adding metadata with 7 columns
299 matching events found
No baseline correction applied
0 projection items activated
Removing 5 compensators from info because not all compensation channels were picked.
Applying baseline correction (mode: mean)
    Using multitaper spectrum estimation with 7 DPSS windows
Reading /home/claraelk/scratch/laughter_data/preproc_bids_data/sub-01/ses-recording/meg/sub-01_ses-recording_task-LaughterActive_proc-clean_epo.fif ...
    Read 5 compensation matrices
    Found the data of interest:
        t =    -500.00 ...    1500.00 ms
        5 CTF compensation matrices available
Adding metadata with 7 columns
299 matching events found
No bas

In [16]:
np.array(all_psd).shape

(12, 75, 270, 76)

In [13]:
spectrum.shape

(76,)

In [4]:
freqs = np.linspace(1, 40, 2401)

In [5]:
freqs.shape

(2401,)

In [7]:
fm = FOOOF()

# Set the frequency range to fit the model
freq_range = [2, 40]

# Report: fit the model, print the resulting parameters, and plot the reconstruction
fm.report(freqs=freqs, power_spectrum=all_subj_psd, freq_range=freq_range)

/home/claraelk/notebook/lib/python3.8/site-packages/fooof/objs/fit.py:1184: RuntimeWarning: invalid value encountered in log10
  power_spectrum = np.log10(power_spectrum)


DataError: The input power spectra data, after logging, contains NaNs or Infs. This will cause the fitting to fail. One reason this can happen is if inputs are already logged. Inputs data should be in linear spacing, not log.

In [ ]:
fooof_path = os.path.join(RESULT_PATH, 'meg', 'reports', 'sub-all')
fm.fit(data_psd, data_psd, freq_range)  
fm.save('fooof_data', fooof_path, save_results=True, save_settings=True, save_data=True)
fm.save_report('fooof_report', fooof_path)